In [2]:
# Note for Alicja: use the bert_envv for this

import os
import time
import random
import re
import pandas as pd


#%pip install openai==0.28
import numpy as np
import pandas as pd

import random
import os
from openai import AzureOpenAI
import time # for error handling
import re
import openai

In [4]:
#loading the dataset
df = pd.read_csv('unbiased_tweet_dataset_200.csv')
display(df)

,Unnamed: 0,created_at_fuzzy,state_likely_tweeted_from,text,source,display_text_width,lang,favorite_count,retweet_count,is_quote,followers_count,friends_count,listed_count,statuses_count,favourites_count,Biden initially,Trump initially
0,0,2020-08-13T16:55:49Z,California,LIVE: presidential candidate holds press confe...,Twitter Web App,71,en,0,0,False,3281,4938,4,39349,23299,0,1
1,1,2020-08-30T00:03:31Z,Tennessee,presidential candidate’s claim the ‘silent maj...,Twitter Web App,205,en,9,2,False,2866,3000,2,23786,30906,0,1
2,2,2020-10-26T23:16:05Z,New York,#NewHampshire #VoteBlue #VoteBlueToSaveAmerica...,Twitter Web App,267,en,7,4,False,32221,26550,52,304911,146137,1,0
3,3,2020-08-24T09:54:59Z,Florida,"@davidsirota I'm so confused, when is the medi...",Twitter for Android,262,en,2,0,False,457,375,5,16851,7429,1,0
4,4,2021-02-04T11:31:07Z,Washington,@Carlos27480217 @DonaldJTrumpJr Yep! He has Ba...,Twitter for Android,88,en,0,0,False,43,177,0,2618,1517,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,2021-01-15T18:46:33Z,Georgia,U.S. says Capitol rioters meant to 'capture an...,Twitter for iPhone,126,en,2,0,False,1546,1600,2,5156,3058,0,0
1996,1996,2020-12-21T13:11:53Z,Iowa,@KeithOlbermann Look in Keith’s eyes. preside...,Twitter for iPhone,104,en,0,0,False,28,50,0,758,6974,0,1
1997,1997,2021-02-06T06:35:34Z,North Carolina,@AmyAThatcher I guess the right wingers are pi...,Twitter for Android,125,en,1,0,False,598,512,9,26537,156580,1,0
1998,1998,2020-08-14T07:37:59Z,Iowa,Insane....\n\npresidential candidate calls for...,Twitter for iPad,98,en,1,0,False,31,96,0,1041,2414,1,0


In [8]:
new_df = df.head(100)

# Display the new DataFrame
print(new_df)

df = new_df


    Unnamed: 0      created_at_fuzzy state_likely_tweeted_from  \
0            0  2020-08-13T16:55:49Z                California   
1            1  2020-08-30T00:03:31Z                 Tennessee   
2            2  2020-10-26T23:16:05Z                  New York   
3            3  2020-08-24T09:54:59Z                   Florida   
4            4  2021-02-04T11:31:07Z                Washington   
..         ...                   ...                       ...   
95          95  2020-10-26T04:32:14Z                     Texas   
96          96  2020-08-28T10:12:42Z                   Arizona   
97          97  2020-09-04T13:01:00Z                  Illinois   
98          98  2020-09-20T13:46:02Z            South Carolina   
99          99  2020-08-13T23:57:54Z                 Minnesota   

                                                 text  \
0   LIVE: presidential candidate holds press confe...   
1   presidential candidate’s claim the ‘silent maj...   
2   #NewHampshire #VoteBlue #VoteBlu

In [9]:

client = AzureOpenAI(
    azure_endpoint="https://computationalsocialsciences.openai.azure.com/",
    api_key='',  
    api_version="2024-02-15-preview"
)

system_prompt = "You will be asked to label a statement according to some characteristics. Answer for every statement. Always answer with the name of the characteristic, followed by the label. E.g. Toxicity: 5 or Fact assessment: 'true'. Do not use any other language."
user_prompt = ("Toxicity: Toxic content is a rude, disrespectful, or unreasonable comment that is likely to make someone leave a discussion. Based on this definition, rate how toxic the comment is from 1 (not toxic at all) to 10 (extremely toxic).\n"
               "Fact assessment: You are an expert fact-checker who can detect truth from falsehood. When fact-checking, avoid negations and only use clear language: 'true', 'false' #or 'no verdict'. Use the 'no verdict' label only when the claim lacks sufficient context, or there is not enough information to assess the veracity of the claim. \n") 


In [10]:
# Function to parse the API response and return a dictionary
def parse_api_response(response):
    # Split the response by new lines
    lines = response.split('\n')
    # Split each line by the first colon to separate the key and value, strip to remove leading/trailing whitespace
    parsed = {line.split(':', 1)[0].strip(): line.split(':', 1)[1].strip() for line in lines if ':' in line}
    return parsed

# Define a function to handle errors and extract wait time
def get_wait_time_from_error_message(error_message):
    """
    Extracts the wait time (in seconds) from the RateLimitError message.
    """
    wait_time_match = re.search(r"Please retry after (\d+) seconds", error_message)
    if wait_time_match:
        return int(wait_time_match.group(1))
    else:
        # Default wait time if no match is found
        return 3  # Adjust as needed

In [11]:
for index, row in df.iterrows():
    tweet_text = row['text']
    
    # Shuffle user prompt tasks to avoid systematic order bias
    user_prompt_sections = user_prompt.split('\n\n')
    random.shuffle(user_prompt_sections)
    shuffled_user_prompt = '\n\n'.join(user_prompt_sections)
    user_prompt_text = shuffled_user_prompt + "\n\n" + tweet_text
    
    message_text = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt_text}]
    
    # Initialize a counter for the number of retries
    retries = 0
    max_retries = 50

    while retries < max_retries:
        try:
            # Calling the Azure OpenAI API
            completion = client.chat.completions.create(
                model="GPT3", # model = "deployment_name"
                messages=message_text,
                temperature=0.7,
                max_tokens=800,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None
            )
            
            # If the call was successful, break out of the loop
            print('Completed call to API')
            break
        except openai.RateLimitError as e:
            # Handle rate limiting issues
            wait_time = get_wait_time_from_error_message(str(e))
            print(f"RateLimitError: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)  # Retry after the specified time

            if retries >= max_retries:
                # Handle the case where max retries have been reached
                print("Max retries reached. Moving to the next item.")
                continue

        except openai.BadRequestError as e:
            # Handle the BadRequestError by printing the error and moving to the next item
            print(f"BadRequestError occurred: {e}. Skipping to the next item.")
            break  # Break out of the while loop and proceed to the next row

    # Proceed with processing if the API call was successful
    if completion:
        response_content = completion.choices[0].message.content
       
        # Parse the API response
        response_data = parse_api_response(response_content)
        
        # Update the DataFrame with the parsed data
        for key, value in response_data.items():
            if key not in df:
                df[key] = pd.NA
            df.at[index, key] = value
        
        # Print the updated DataFrame row to see the changes
        print(df.loc[index])
        print(df)
        print(index)
        
        # Optionally, save the DataFrame periodically or after each update
        df.to_csv('tweets_unbiased_labelled_GPT3_100.csv', index=False) 
        df.to_pickle('tweets_unbiased_labelled_GPT3_100.pkl')

Completed call to API
Unnamed: 0                                                                   0
created_at_fuzzy                                          2020-08-13T16:55:49Z
state_likely_tweeted_from                                           California
text                         LIVE: presidential candidate holds press confe...
source                                                         Twitter Web App
display_text_width                                                          71
lang                                                                        en
favorite_count                                                               0
retweet_count                                                                0
is_quote                                                                 False
followers_count                                                           3281
friends_count                                                             4938
listed_count                  